# 为 vllm 增加对 baichuan-inc/baichuan-7B 的支持

### 拉取代码

In [1]:
!rm -fr vllm && git clone https://github.com/gameofdimension/vllm.git

Cloning into 'vllm'...
remote: Enumerating objects: 2253, done.
remote: Counting objects: 100% (1511/1511), done.
remote: Compressing objects: 100% (794/794), done.
remote: Total 2253 (delta 1015), reused 982 (delta 710), pack-reused 742
Receiving objects: 100% (2253/2253), 3.08 MiB | 30.02 MiB/s, done.
Resolving deltas: 100% (1358/1358), done.


In [2]:
%cd /content/vllm

/content/vllm


In [3]:
!git checkout baichuan && git pull

Branch 'baichuan' set up to track remote branch 'baichuan' from 'origin'.
Switched to a new branch 'baichuan'
Already up to date.


### 安装依赖

In [4]:
!pip install -r requirements.txt

### 从源码安装 vllm

In [5]:
!pip install -e .

Obtaining file:///content/vllm
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for vllm (pyproject.toml) ... done
  Created wheel for vllm: filename=vllm-0.1.2-0.editable-cp310-cp310-linux_x86_64.whl size=8342 sha256=31dfb83e037dc56e54192400f0fae5971cb1cf5101462d4c7d702b3f234766fb
  Stored in directory: /tmp/pip-ephem-wheel-cache-cfrusn8s/wheels/35/28/89/44ebc8017c7395bab306eb7218f34e5d09408126dd82c20f16
Successfully built vllm
  Attempting uninstall: vllm
    Found existing installation: vllm 0.1.2
    Uninstalling vllm-0.1.2:
      Successfully uninstalled vllm-0.1.2


### 验证语言模型功能正常

In [6]:
from vllm import LLM, SamplingParams
llm = LLM(model="baichuan-inc/baichuan-7B")

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.


INFO 07-07 11:55:38 llm_engine.py:60] Initializing an LLM engine with config: model='baichuan-inc/baichuan-7B', tokenizer='baichuan-inc/baichuan-7B', tokenizer_mode=auto, dtype=torch.float16, use_dummy_weights=False, download_dir=None, use_np_weights=False, tensor_parallel_size=1, seed=0)


Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


WARNING 07-07 11:55:40 tokenizer.py:46] Using a slow tokenizer. This might cause a significant slowdown. Consider using a fast tokenizer instead.


INFO 07-07 11:59:30 llm_engine.py:131] # GPU blocks: 2743, # CPU blocks: 512


In [7]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
    "希望这篇文章能",
    "给六岁小朋友解释一下万有引",
]
sampling_params = SamplingParams(temperature=0.1, top_p=0.95)
outputs = llm.generate(prompts, sampling_params)

# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Processed prompts: 100%|██████████| 6/6 [00:00<00:00, 15.27it/s]

Prompt: 'Hello, my name is', Generated text: 'Kaitlyn and I am a senior at the University of Wisconsin-Mad'
Prompt: 'The president of the United States is', Generated text: 'a man who has been accused of sexual assault by 16 women. He'
Prompt: 'The capital of France is', Generated text: 'Paris, which is located in the north-central part of the country.'
Prompt: 'The future of AI is', Generated text: 'in the cloud\nBy: Chris Preimesberger, eWEEK |'
Prompt: '希望这篇文章能', Generated text: '帮助到你,如果你有更好的建议,欢迎在下方留言,我们一起'
Prompt: '给六岁小朋友解释一下万有引', Generated text: '力,为什么月亮会绕着地球转?\n因为月亮的质量比'


# 以下验证 vllm 对 baichuan-inc/baichuan-7B 的加速能力

### 下载测试数据

In [6]:
!wget https://huggingface.co/datasets/anon8231489123/ShareGPT_Vicuna_unfiltered/resolve/main/ShareGPT_V3_unfiltered_cleaned_split.json

--2023-07-07 12:19:35--  https://huggingface.co/datasets/anon8231489123/ShareGPT_Vicuna_unfiltered/resolve/main/ShareGPT_V3_unfiltered_cleaned_split.json
Resolving huggingface.co (huggingface.co)... 13.33.33.102, 13.33.33.55, 13.33.33.110, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.102|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/58/74/5874e8234cbcd37dd31ca486e8492d9f1370bdd04829001f53991a866851e83f/35f0e213ce091ed9b9af2a1f0755e9d39f9ccec34ab281cd4ca60d70f6479ba4?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27ShareGPT_V3_unfiltered_cleaned_split.json%3B+filename%3D%22ShareGPT_V3_unfiltered_cleaned_split.json%22%3B&response-content-type=application%2Fjson&Expires=1688991575&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY4ODk5MTU3NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy81OC83NC81ODc0ZTgyMzRjYmNkMzdkZDMxY2E0ODZlO

### 测试 vllm 吞吐。因为模型较大，上面一步的显存如果没有释放，可能需要重启 kernel，并且跳过上面调用 python api 的测试，直接执行运行以下的 cells

In [7]:
!python3 benchmarks/benchmark_throughput.py --model "baichuan-inc/baichuan-7B" --dataset ./ShareGPT_V3_unfiltered_cleaned_split.json

Namespace(backend='vllm', dataset='./ShareGPT_V3_unfiltered_cleaned_split.json', model='baichuan-inc/baichuan-7B', tokenizer='baichuan-inc/baichuan-7B', tensor_parallel_size=1, n=1, use_beam_search=False, num_prompts=1000, seed=0, hf_max_batch_size=None)
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
WARNING 07-07 12:19:42 tokenizer.py:46] Using a slow tokenizer. This might cause a significant slowdown. Consider using a fast tokenizer instead.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
INFO 07-07 12:25:42 llm_engine.py:60] Initializing an LLM engine with config: model='baichuan-inc/baichuan-7B', tokenizer='baichuan-inc/baichuan-7B', tokenizer_mode=auto, dtype=torch.float16, use_dummy_weights=False, download_dir=None, use_np_weights=False, tensor_parallel_size=1, s

### 测试 huggingface 吞吐

In [11]:
!python3 benchmarks/benchmark_throughput.py --num-prompts 100 --model "baichuan-inc/baichuan-7B" --dataset ./ShareGPT_V3_unfiltered_cleaned_split.json --backend hf --hf-max-batch-size 4

Namespace(backend='hf', dataset='./ShareGPT_V3_unfiltered_cleaned_split.json', model='baichuan-inc/baichuan-7B', tokenizer='baichuan-inc/baichuan-7B', tensor_parallel_size=1, n=1, use_beam_search=False, num_prompts=100, seed=0, hf_max_batch_size=4)
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
WARNING 07-07 13:13:34 tokenizer.py:46] Using a slow tokenizer. This might cause a significant slowdown. Consider using a fast tokenizer instead.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
2023-07-07 13:19:35.162808: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
100% 100/10

In [12]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained("baichuan-inc/baichuan-7B", trust_remote_code=True)
print(config.model_type)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.


baichuan
